---

---

# 01. Introduction

In this session, you need an account of GEM machine.  
Here we want to propose what you can do with the NewCluster (NC) simulation raw data.  

---

## I. Particle

### Dark Matter

The dark matter in NC is treated as a cold dark matter (CDM).

The gravitation is the one and only source dertermine the movement of the DM particles.



The mass of the DM particle ($M_{DM}\, \sim\, 10^6\ \rm M_{\odot}$) inside the target zoom-in region is larger than the stellar particles ($M_{*}\, \sim\, 2\times 10^4\ \rm M_{\odot}$)




### Star

data dype:

- 'x', 'y', 'z' : position
- 'vx', 'vy', 'vz' : position
- 'm' : current mass
- 'm0' : initial mass
- 'rho0' : background gas density at birth position
- 'epoch' : related to birth time (you can directly use 'age')
- 'metal' : total metal fraction
- 'H' ~ 'D' : element abundance

- 'id' : identifier number
- 'family' : particle type indicator

### SMBH

---

## II. AMR/Hydro

data dype:

('x', 'y', 'z', 'rho', 'vx', 'vy', 'vz', 'P', 'metal', 'H', 'O', 'Fe', 'Mg', 'C', 'N', 'Si', 'S', 'D', 'd1', 'd2', 'd3', 'd4', 'refmask', 'sigma', 'level', 'cpu')

- 'x', 'y', 'z' : position
- 'vx', 'vy', 'vz' : position
- 'm' : mass
- 'level' : AMR level
- 'rho' : mass density
- 'P' : pressure
- 'T' : temperature
- 'metal' : total metal fraction
- 'H' ~ 'D' : element abundance
- 'd1' : fraction of small Carbonaceous dust (a = 0.005 micron)
- 'd2' : fraction of small Carbonaceous dust (a = 0.1 micron)
- 'd3' : Silicon fraction of small Silicate dust (a = 0.005 micron)
- 'd4' : Silicon fraction of small Silicate dust (a = 0.1 micron)
- 'sigma' : internal gas velocity dispersion

** For silicate dust (d3, d4), you should divide 0.163 (the mass fraction of Si in the Silicate dust) to get the Silicate dust fraction

---

## III. Tracer

please find the detailed description for the tracer particles at 

**Accurate tracer particles of baryon dynamics in the adaptive mesh refinement code Ramses**
https://ui.adsabs.harvard.edu/abs/2019A%26A...621A..96C/abstract

---

---

# 02. Access data with `RUR`

---

## Preparation

Install `RUR`
```bash
guest@GC11:~$ git clone https://github.com/sanhancluster/rur.git
guest@GC11:~$ cd rur
guest@GC11:~/rur$ ./f2py        # Compile fortran-based functions
guest@GC11:~$ conda develop .   # If you use conda, add this repository to PATH
```

In [1]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('classic')
from matplotlib import colors
from matplotlib import gridspec
import scipy

import rur
from rur import uri, uhmi, painter, vrload
from rur.sim import NewCluster
from rur.sci import photometry, chemistry

### Get snapshot information

In [2]:
# Setting
NC = NewCluster(411)
print(f"\n > Parameters: {NC.params}\n")

# Get time series
NCs = uri.TimeSeries(NC)
NCs.read_iout_avail()
iout_avail = NCs.iout_avail
first = iout_avail[0]
print(f" > {first['iout']}th snapshot: z={1/first['aexp']-1:.2f}  age={first['age']:.2f} Gyr")
last = iout_avail[-1]
print(f" > {last['iout']}th snapshot: z={1/last['aexp']-1:.2f}  age={last['age']:.2f} Gyr")

[Output 00411] Age (Gyr) : 5.327 / 13.761, z = 1.17316 (a = 0.4602)

 > Parameters: {'nstar': array([624161037], dtype=int32), 'star': array([ True]), 'ncpu': 960, 'ndim': 3, 'levelmin': 8, 'levelmax': 21, 'ngridmax': 4000000, 'nstep_coarse': 38262, 'boxlen': 1.0, 'time': -1.34385025856362, 'aexp': 0.460158685234279, 'H0': 70.3000030517578, 'omega_m': 0.272000014781952, 'omega_l': 0.727999985218048, 'omega_k': 0.0, 'omega_b': 0.0455, 'unit_l': 2.01605799705751e+26, 'unit_d': 2.5936692558499e-29, 'unit_t': 9.27706542995716e+16, 'unit_m': 2.12531671957152e+50, 'h': 0.703000030517578, 'z': 1.173163371872191, 'boxsize': 100.00000585166977, 'boxsize_physical': 65.45642847589318, 'boxsize_comoving': 142.24751281738293, 'icoarse': 38262, 'ordering': 'hilbert', 'age': 5.327376358967498, 'lookback_time': 8.433483869839588}

 > 1th snapshot: z=50.00  age=0.05 Gyr
 > 521th snapshot: z=0.99  age=6.01 Gyr


---

### Get catalogue

In [3]:
# Get HaloMaker and GalaxyMaker (VR NOT SUPPORTED YET)
HM = uhmi.HaloMaker()
mgals = HM.load(NC, galaxy=True) # GalaxyMaker
print(f" > {len(mgals)} galaxies")
mhals = HM.load(NC, galaxy=False) # HaloMaker
print(f" > {len(mhals)} halos")

 > 7411 galaxies
 > 99150 halos


In [4]:
import zipfile, os
zname_HM = "./gm_example.zip" # <- Change this to the name of the zip file you want to extract
with zipfile.ZipFile(zname_HM, 'r') as zip_ref:
    zip_ref.extractall()
    
if(np.__version__ >= '1.16.3'):
    HMs = np.load("./GalaxyMaker_00411.pkl", allow_pickle=True)
else:
    import pickle as pkl
    with open("./GalaxyMaker_00411.pkl", 'rb') as f:
        HMs = pkl.load(f)

---

### Get Raw data near target

In [5]:
target = mgals[40]
print(f" > Available: {target.dtype.names}")
print(f"Target galaxy: {target['id']}")
print(f" > Nparts = {target['nparts']} stars")
print(f" > M* = {target['m']:.2e} Msol")
print(f" > Rmax = {target['r']/NC.unit['kpc']:.2f} kpc")
print(f" > R50 = {target['r50']/NC.unit['kpc']:.2f} kpc")
print(f" > R90 = {target['r90']/NC.unit['kpc']:.2f} kpc")
print(f" > age(rband) = {target['ager']:.2f} Gyr")

# Set 10 kpc box
NC.set_box_halo(target, radius=10*NC.unit['kpc'], use_halo_radius=False)
print(f"\n > Box: \n{NC.box}")

 > Available: ('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'Mcold_gas_r90', 'Mdense_gas_r50', 'Mdense_gas_r90', 'O_gas', 'Mg_gas', 'C_gas', 'Si_gas', 'D_gas', 'CDustSmall_gas', 'SiDustSmall_gas', 'SBu', 'SBu_r50', 'SBg', 'SBg_r50', 'SBr', 'SBr_r50', 'SBi', 'SBi_r90', 'SBz', 'SBz_r90', 'MBH', 'r50', 'r90', 'age', 'r50u', 'r90u', 'r50g', 'r90g', 'r50r', 'r90r', 'r50i', 'r90i', 'r50z', 'r90z', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ageu', 'ageg', 'ager', 'agei', 'agez', 'SFR10', 'vsig', 'SFR10_r50', 'SFR10_r90', 'vsig_r50', 'vsig_r90', 'SFR', 'SFR_r50', 'SFR_r90', 'metal', 'vsig_gas', 'vsig_gas_r50', 'vsig_gas_r90', 'metal_gas', 'M_gas', 'M_gas_r50', 'M_gas_r90', 'Mcold_gas', 'Mcold_gas_r50', 'Mdense_gas', 'H_gas', 'Fe_gas', 'N_gas', 'S_gas', 'CDustLarge_gas', 'SiDust

---

### Get Particle

In [6]:
part = NC.get_part(nthread=5)
print(f"\n{len(part)} particles")
star = part['star']
print(f" > {len(star)} stars")
dm = part['dm']
print(f" > {len(dm)} dark matters")
print(f" > Available: {part.dtype.names}")

Reading 5 part files (832.5 MiB) in /storage7/NewCluster/snapshots/output_00411... 
	Allocating Memory...
	Done (0.238 sec) -> 5017177 particles


Reading parts:   0%|          | 0/5 [00:00<?, ?it/s]

Done (3.337s).
	Masking...
	Done (2.595s).
Masking particles...
Done (11.300s).

3457557 particles
 > 3204486 stars
 > 89635 dark matters
 > Available: ('x', 'y', 'z', 'vx', 'vy', 'vz', 'm', 'id', 'level', 'family', 'tag', 'epoch', 'metal', 'm0', 'H', 'O', 'Fe', 'Mg', 'C', 'N', 'Si', 'S', 'D', 'rho0', 'partp', 'cpu')


---

### Get Cell

In [7]:
cell = NC.get_cell(nthread=5)
print(f"\n{len(cell)} cells")
lvls, counts = np.unique(cell['level'], return_counts=True)
for lvl, count in zip(lvls, counts):
    print(f" > {count} cells at level {lvl} ({(1 / 2**lvl)/NC.unit['pc']:.2f} pc)")
print(f" > Available: {cell.dtype.names}")

Reading 5 AMR & hydro files (645.4 MiB) in /storage7/NewCluster/snapshots/output_00411... 
	Allocating Memory...
	Done (0.465 sec) -> 2599102 cells


Reading cells:   0%|          | 0/5 [00:00<?, ?it/s]

Done (2.480s).
	Masking...
	Done (2.468s).
Masking cells...
Done (6.954s).

625337 cells
 > 1390 cells at level 16 (998.79 pc)
 > 44463 cells at level 17 (499.39 pc)
 > 94597 cells at level 18 (249.70 pc)
 > 212431 cells at level 19 (124.85 pc)
 > 272456 cells at level 20 (62.42 pc)
 > Available: ('x', 'y', 'z', 'rho', 'vx', 'vy', 'vz', 'P', 'metal', 'H', 'O', 'Fe', 'Mg', 'C', 'N', 'Si', 'S', 'D', 'd1', 'd2', 'd3', 'd4', 'refmask', 'sigma', 'level', 'cpu')


---

### Get SMBHs

In [ ]:
smbh = NC.get_sink()
clouds = part['sink']
print(f"\n{len(smbh)} SMBHs have {len(clouds)} clouds")
print(f" > Available: {smbh.dtype.names}")

Reading a sink file (625.9 KiB) in /storage7/NewCluster/snapshots/output_00411... 
Done (0.047s).
Masking sinks... 1 / 675 (0.0015)
Done (0.000s).


ㅤ  
ㅤ  
ㅤ  
ㅤ  
ㅤ  
ㅤ  
ㅤ  
ㅤ  

---

---

# 03. Visualize Data

---

## I. Particle density map

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(9,6))
# Remove ticks
for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])


###### X-Y projection ######
# Dark Matter density map
ax = axes[0,0]
im = painter.draw_partmap(dm, ax=ax, cmap=plt.cm.bone, qscale=2, proj=[0,1])
ax.set_title('Dark Matter')

# Stellar density map
ax = axes[0,1]
im = painter.draw_partmap(star, ax=ax, cmap=plt.cm.afmhot, qscale=4, proj=[0,1])
ax.set_title('Star (Density)')

# u-band image
ax = axes[0,2]
uband = photometry.measure_luminosity(star, 'SDSS_u')
im = painter.draw_partmap(star, ax=ax, cmap=plt.cm.gray_r, qscale=4, weights=uband, proj=[0,1])
ax.set_title('u-band Image')

###### X-Z projection ######
# Dark Matter density map
ax = axes[1,0]
im = painter.draw_partmap(dm, ax=ax, cmap=plt.cm.bone, qscale=2, proj=[0,2])

# Stellar density map
ax = axes[1,1]
im = painter.draw_partmap(star, ax=ax, cmap=plt.cm.afmhot, qscale=4, proj=[0,2])

# u-band image
ax = axes[1,2]
im = painter.draw_partmap(star, ax=ax, cmap=plt.cm.gray_r, qscale=4, weights=uband, proj=[0,2])

plt.subplots_adjust(wspace=0, hspace=0)
plt.show(); plt.close()

---

## II. Gas maps

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(9,6))
# Remove ticks
for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])


###### X-Z projection ######
# Cell density map
ax = axes[0,0]
painter.draw_gasmap(cell, ax=ax, mode='rho', unit='H/cc', cmap=plt.cm.bone, qscale=2, proj=[0,2])
ax.set_title('Density')

# Cell temperature map
ax = axes[0,1]
vmax = 1e7 # Kelvin
vmin = 1e3 # Kelvin
painter.draw_gasmap(cell, ax=ax, mode='T', unit='K', cmap=plt.cm.coolwarm, proj=[0,2], vmax=vmax, vmin=vmin)
ax.set_title('Temperature')

# Cell metallicity map
ax = axes[0,2]
painter.draw_gasmap(cell, ax=ax, mode='metal', cmap=plt.cm.winter, proj=[0,2], normmode='linear')
ax.set_title('Metallicity')

# Cell AMR level map
ax = axes[1,0]
# painter.draw_grid(cell, ax=ax, proj=[0,2], cmap=plt.cm.jet) # <- slow but detailed
painter.draw_lvlmap(cell, ax=ax, proj=[0,2], cmap=plt.cm.jet, qscale=0.1)
ax.set_title('AMR level')

# Chemical Map
ax = axes[1,1]
FeH = chemistry.Z_over_Z(cell, chem1='Fe', chem2='H')
painter.draw_gasmap(cell, quantity=FeH, ax=ax, proj=[0,2], cmap=plt.cm.jet, normmode='log', qscale=1)
ax.set_title('[Fe/H]')

# Dust-to-Metal
ax = axes[1,2]
muMg=24.3050; muFe=55.8450; muSi=28.0855; muO =15.9990
nsilMg=1.0; nsilFe=1.0; nsilSi=1.0; nsilO=4.0
numtot=muMg*nsilMg+muFe*nsilFe+muSi*nsilSi+muO*nsilO
SioverSil=muSi*nsilSi/numtot
# d1: Carbonaceous 0.005 um
# d2: Carbonaceous 0.1 um
# d3: Silicate 0.005 um
# d4: Silicate 0.1 um
mdust = cell['m','Msol'] * (cell['d1'] + cell['d2'] + (cell['d3']+cell['d4'])/SioverSil)
mmetal = cell['m','Msol'] * cell['metal']
DTM = mdust / mmetal
painter.draw_gasmap(cell, quantity=DTM, ax=ax, proj=[0,2], cmap=plt.cm.BrBG, vmax=0.4, vmin=0.004, normmode='log')
ax.set_title('Dust-to-Metal')


plt.subplots_adjust(wspace=0, hspace=0.2)
plt.show(); plt.close()

---

## III. Other maps

### SMBH Map

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
# Remove ticks
ax.set_xticks([]); ax.set_yticks([])

# Set 10 kpc box
NC.set_box_halo(target, radius=10*NC.unit['kpc'], use_halo_radius=False)

# Background image
painter.draw_partmap(star, box=NC.box, ax=ax, cmap=plt.cm.gray, qscale=3, proj=[0,2], weights=uband)


# Set 1 kpc box
NC.set_box_halo(target, radius=1*NC.unit['kpc'], use_halo_radius=False)

# Inset axis
axin = ax.inset_axes([0.02, 0.02, 0.4, 0.4])
for spine in axin.spines.values(): spine.set_edgecolor('orange')
# Remove ticks
axin.set_xticks([]); axin.set_yticks([])

# Draw box
x1 = NC.box[0,0]; x2 = NC.box[0,1]
z1 = NC.box[2,0]; z2 = NC.box[2,1]
rectangle = plt.Rectangle((x1, z1), x2-x1, z2-z1, ec='orange', fc='none')
ax.add_patch(rectangle)

# Background image
vmax = 1e7; vmin = 5e3
painter.draw_gasmap(cell, 
                    box=NC.box, 
                    ax=axin, 
                    mode='T', unit='K', 
                    cmap=plt.cm.bwr, 
                    proj=[0,2], 
                    vmax=vmax, 
                    vmin=vmin)

axin.scatter(clouds['x'], clouds['z'], ec='none', fc='orange', s=1)
axin.scatter(smbh['x'], smbh['z'], color='magenta', s=20, marker='x')

ax.text(0.985, 0.015, 
        fr"$\rm MBH={np.log10(smbh[0]['m','Msol']):.2f}\ M_\odot$", 
        size=15,
        color='w', 
        ha='right', va='bottom', 
        transform=ax.transAxes)

plt.show(); plt.close()

---

### Tracking gas tracers

In [ ]:
iout_avail = iout_avail[iout_avail['iout']<=411]

In [ ]:
# Get gas tracers
gas_tracer = part['gas_tracer']
print(f" > {len(gas_tracer)} gas tracers")

# Choose central tracers
distances = np.sqrt((target['x']-gas_tracer['x'])**2 + (target['y']-gas_tracer['y'])**2 + (target['z']-gas_tracer['z'])**2)
mask = distances < 0.5*target['r50']
target_tracers = gas_tracer[mask]
#print(f" > {len(target_tracers)} gas tracers in 0.5 R50 (={0.5*target['r50']/NC.unit['pc']:.2f} pc)")



# See motions during last 8 snapshots
lengt = 8
target_iouts = iout_avail[-lengt:]['iout']
target_iouts = target_iouts[target_iouts != last['iout']][::-1]

# Save tracers in previous 8 snapshots
tmp_dictionary = {}
for iout in target_iouts:
    iNC = NCs.get_snap(iout)
    iNC.set_box_halo(target, radius=20*NC.unit['kpc'], use_halo_radius=False)
    uri.timer.verbose=0 # No verbose mode
    iNC.get_part(pname='tracer', target_fields=['x','y','z','id','family'], nthread=5)
    uri.timer.verbose=1
    tmp_dictionary[iout] = iNC.part
    print(f" > {iout}th snapshot: {len(iNC.part['tracer'])} tracers")

In [ ]:
from rur.drawer import colorline
from scipy.ndimage import gaussian_filter1d
fig, ax = plt.subplots(figsize=(6,6))
# Remove ticks
ax.set_xticks([]); ax.set_yticks([])


# Background image
NC.set_box_halo(target, radius=10*NC.unit['kpc'], use_halo_radius=False)
painter.draw_gasmap(cell, ax=ax, box=NC.box, mode='rho', unit='H/cc', cmap=plt.cm.gray, qscale=2, proj=[0,2])
ax.set_xlim(*NC.box[0])
ax.set_ylim(*NC.box[2])

# Inset axis
axin = ax.inset_axes([0.52, 0.02, 0.4, 0.4])
for spine in axin.spines.values(): spine.set_edgecolor('dodgerblue')
# Remove ticks
axin.set_xticks([]); axin.set_yticks([])
#NC.set_box_halo(target, radius=0.5, use_halo_radius=True, radius_name='r50')
NC.set_box_halo(target, radius=5*NC.unit['kpc'], use_halo_radius=True)
painter.draw_gasmap(cell, ax=axin, box=NC.box, mode='rho', unit='H/cc', cmap=plt.cm.gray, qscale=1, proj=[0,2])

# Draw tracer tracks
colorss = plt.cm.bwr_r(np.linspace(0, 1, len(target_tracers)))
for target_tracer, color in zip(target_tracers, colorss):
    xs = np.zeros(lengt); zs = np.zeros(lengt); ts = np.zeros(lengt)
    xs[0] = target_tracer['x']; zs[0] = target_tracer['z']; ts[0] = 411
    for i, iout in enumerate(target_iouts):
        if i >= lengt-1: break
        tmp = tmp_dictionary[iout]
        # Out of box
        if(target_tracer['id'] not in tmp['id']): break
        idx = np.where(tmp['id'] == target_tracer['id'])[0][0]
        xs[i+1] = tmp[idx]['x']; zs[i+1] = tmp[idx]['z']; ts[i+1] = iout
    mask = xs>0
    xs = xs[mask]; zs = zs[mask]; ts = ts[mask]
    axin.scatter(xs[0], zs[0], marker='*', ec='none', fc=colorss[-1], s=20)
    colorline(xs, zs, z=ts, cmap='bwr_r', norm=plt.Normalize(target_iouts[-1], 411), linewidth=0.2, ax=ax, zorder=2)
    colorline(xs, zs, z=ts, cmap='bwr_r', norm=plt.Normalize(target_iouts[-1], 411), linewidth=0.3, ax=axin, zorder=2)



# Draw box
x1 = NC.box[0,0]; x2 = NC.box[0,1]
z1 = NC.box[2,0]; z2 = NC.box[2,1]
rectangle = plt.Rectangle((x1, z1), x2-x1, z2-z1, ec='dodgerblue', fc='none')
ax.add_patch(rectangle)

axin.set_xlim(x1, x2)
axin.set_ylim(z1, z2)

plt.show(); plt.close()

---

---

# ✏️04. Examples

---

## get magnitude

In [ ]:
print("Measuring the luminosity of the stellar particles")
print("using the saved table (w/ Bruzual & Charlot 03 model)")

In [ ]:
sdss_u = rur.sci.photometry.measure_magnitude(star,'SDSS_u')
sdss_g = rur.sci.photometry.measure_magnitude(star,'SDSS_g')
sdss_r = rur.sci.photometry.measure_magnitude(star,'SDSS_r')
sdss_i = rur.sci.photometry.measure_magnitude(star,'SDSS_i')
sdss_z = rur.sci.photometry.measure_magnitude(star,'SDSS_z')

## unit conversion

In [ ]:
print("Minmax of the coordinates of the loaded particles")
print("       (*** Before unit conversion ***)")
print('--------------------------------------------------')
print("   X = [%s, %s]"%(np.min(part['x']),np.max(part['x'])))
print("   Y = [%s, %s]"%(np.min(part['y']),np.max(part['y'])))
print("   Z = [%s, %s]"%(np.min(part['z']),np.max(part['z'])))
print('--------------------------------------------------')

In [ ]:
NC.switch_unitmode()

In [ ]:
print("Minmax of the coordinates of the loaded particles")
print("       (*** After unit conversion ***)")
print('--------------------------------------------------')
print("   X = [%s, %s]"%(np.min(part['x']),np.max(part['x'])))
print("   Y = [%s, %s]"%(np.min(part['y']),np.max(part['y'])))
print("   Z = [%s, %s]"%(np.min(part['z']),np.max(part['z'])))
print('--------------------------------------------------')

In [ ]:
print('-----------------------')
print(f"{len(part)} particles")
star = part['star']
print(f" > {len(star)} stars")
dm = part['dm']
print(f" > {len(dm)} dark matters")
print('----------------------')

In [ ]:
import copy
ktrg = copy.deepcopy(target)

In [ ]:
ktrg['x'] /= NC.unit['kpc']
ktrg['y'] /= NC.unit['kpc']
ktrg['z'] /= NC.unit['kpc']

In [ ]:
print('galaxy center position in sim unit')
print('-------------------------------------')
print('    X =',target['x'],'[kpc]')
print('    Y =',target['y'],'[kpc]')
print('    Z =',target['z'],'[kpc]')
print('-------------------------------------')

In [ ]:
print('galaxy center position in kpc unit')
print('-------------------------------------')
print('    X =',ktrg['x'],'[kpc]')
print('    Y =',ktrg['y'],'[kpc]')
print('    Z =',ktrg['z'],'[kpc]')
print('-------------------------------------')

In [ ]:
print("Minmax of the coordinates of the loaded particles")
print("   (*** After changing the center position ***)")
print('--------------------------------------------------')
print("   X = [%s, %s]"%(np.min(part['x']-ktrg['x']),np.max(part['x']-ktrg['x'])))
print("   Y = [%s, %s]"%(np.min(part['y']-ktrg['y']),np.max(part['y']-ktrg['y'])))
print("   Z = [%s, %s]"%(np.min(part['z']-ktrg['z']),np.max(part['z']-ktrg['z'])))
print('--------------------------------------------------')

## ❓QUIZ 01❓
 > **1. Draw XY(f1) & XZ(f2) histogram of the stellar particles**  
 > **2. Try changing the weight array (mass, flux)**  

In [ ]:
fig = plt.figure(figsize=(16,8))
f1 = fig.add_subplot(1,2,1)
f2 = fig.add_subplot(1,2,2)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 

yarr1 = 
yarr2 = 

warr = 

dlim = 10
bins = 300

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr1,
          weights=warr,
          range=[[-dlim,dlim],[-dlim,dlim]],
          bins=bins,
          norm=colors.LogNorm())

f2.hist2d(xarr,
          yarr2,
          weights=warr,
          range=[[-dlim,dlim],[-dlim,dlim]],
          bins=bins,
          norm=colors.LogNorm())

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

lab_size = 20
f_tick = 4/5
f_legend = 4/5

f1.set_title("Face-on",size=lab_size,pad=8)
f1.set_xlabel('$X\ (\mathrm{kpc})$',size=lab_size)
f1.set_ylabel('$Y\ (\mathrm{kpc})$',size=lab_size)

f2.set_title("Edge-on",size=lab_size,pad=8)
f2.set_xlabel('$X\ (\mathrm{kpc})$',size=lab_size)
f2.set_ylabel('$Z\ (\mathrm{kpc})$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f2.tick_params(axis='x',labelsize=f_tick*lab_size)
f2.tick_params(axis='y',labelsize=f_tick*lab_size)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.tight_layout()
plt.show()

## ❓QUIZ 02❓
 > **1. Rotate galaxy with respect to its net angular momentum direction**
>
 > **2. Draw spatial distribution again**

In [ ]:
star = rur.sci.kinematics.align_axis(part=star, 
                                     gal=ktrg,
                                     prefix=None)

In [ ]:
fig = plt.figure(figsize=(16,8))
f1 = fig.add_subplot(1,2,1)
f2 = fig.add_subplot(1,2,2)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 

yarr1 = 
yarr2 = 

warr = 

dlim = 10
bins = 300

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr1,
          weights=warr,
          range=[[-dlim,dlim],[-dlim,dlim]],
          bins=bins,
          norm=colors.LogNorm())

f2.hist2d(xarr,
          yarr2,
          weights=warr,
          range=[[-dlim,dlim],[-dlim,dlim]],
          bins=bins,
          norm=colors.LogNorm())

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

lab_size = 20
f_tick = 4/5
f_legend = 4/5

f1.set_title("Face-on",size=lab_size,pad=8)
f1.set_xlabel('$X\ (\mathrm{kpc})$',size=lab_size)
f1.set_ylabel('$Y\ (\mathrm{kpc})$',size=lab_size)

f2.set_title("Edge-on",size=lab_size,pad=8)
f2.set_xlabel('$X\ (\mathrm{kpc})$',size=lab_size)
f2.set_ylabel('$Z\ (\mathrm{kpc})$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f2.tick_params(axis='x',labelsize=f_tick*lab_size)
f2.tick_params(axis='y',labelsize=f_tick*lab_size)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.tight_layout()
plt.show()

## ❓QUIZ 03❓
 > **Draw Age or Metallicity 1d histogram of the stellar particles**  

In [ ]:
fig = plt.figure(figsize=(10,8))
f1 = fig.add_subplot(1,1,1)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 
warr = None

bins = 200

xmin = 0
xmax = 5.5

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='stepfilled',
        color='c',
        alpha=0.5)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

lab_size = 20
f_tick = 4/5
f_legend = 4/5

f1.set_xlabel('$Age\ (\mathrm{Gyr})$',size=lab_size)
f1.set_ylabel('$N$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(1e-1,)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,8))
f1 = fig.add_subplot(1,1,1)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Zsol = 0.0134

xarr = 
warr = None

bins = 200
xmin = -2
xmax = 0.5

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='stepfilled',
        color='c',
        alpha=0.5)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

lab_size = 20
f_tick = 4/5
f_legend = 4/5

f1.set_xlabel('$log\ Z/Z_{\odot}$',size=lab_size)
f1.set_ylabel('$N$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(1e-1,)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.tight_layout()
plt.show()

## ❓QUIZ 03-2❓
 > **Draw Age & Metallicity 2d histogram of the stellar particles**  

In [ ]:
fig = plt.figure(figsize=(15,8))

f2 = plt.subplot2grid((4,6), (0, 0),fig=fig,colspan=4)
f1 = plt.subplot2grid((4,6), (1, 0),colspan=4,rowspan=3,fig=fig)
f3 = plt.subplot2grid((4,6), (1, 4),fig=fig,rowspan=3)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 
yarr = 

warr = None

bins = 200
cmap=plt.cm.magma

xmin = 0
xmax = 5.5

ymin = -2
ymax = 0.5

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr,
          weights=warr,
          range=[[xmin,xmax],[ymin,ymax]],
          bins=bins,
          cmap=cmap,
          norm=colors.LogNorm())


f2.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2)
f3.hist(yarr,
        weights=warr,
        range=[ymin,ymax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2,
        orientation='horizontal')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.set_xlabel('$Age\ (\mathrm{Gyr})$',size=lab_size)
f1.set_ylabel('$log\ Z/Z_{\odot}$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(ymin,ymax)

f2.set_xlim(xmin,xmax)
f2.set_xticks([])
f2.set_yticks([])

f3.set_ylim(ymin,ymax)
f3.set_xticks([])
f3.set_yticks([])

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.subplots_adjust(wspace=0,hspace=0)

#plt.tight_layout()
plt.show()

## ❓QUIZ 04❓
 > **Draw [Mg/Fe] vs. [Fe/H] diagram**  

In [ ]:
from rur.sci import chemistry as chem

In [ ]:
solar_H = chem.solar_frac['H']
solar_Mg = chem.solar_frac['Mg']
solar_Fe = chem.solar_frac['Fe']

In [ ]:
FeH = 
MgFe = 

In [ ]:
fig = plt.figure(figsize=(14,8))

f2 = plt.subplot2grid((4,6), (0, 0),fig=fig,colspan=4)
f1 = plt.subplot2grid((4,6), (1, 0),colspan=4,rowspan=3,fig=fig)
f3 = plt.subplot2grid((4,6), (1, 4),fig=fig,rowspan=3)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 
yarr = 

warr = None

bins = 200
cmap=plt.cm.magma

xmin = -3
xmax = 0.5

ymin = -0.2
ymax = 1

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr,
          weights=warr,
          range=[[xmin,xmax],
                 [ymin,ymax]],
          bins=bins,
          cmap=cmap,
          norm=colors.LogNorm())


f2.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2)
f3.hist(yarr,
        weights=warr,
        range=[ymin,ymax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2,
        orientation='horizontal')

f1.axvline(0,
           ls='--',
           lw=2,
           color='k')
f1.axhline(0,
           ls='--',
           lw=2,
           color='k')
f2.axvline(0,
           ls='--',
           lw=2,
           color='k')
f3.axhline(0,
           ls='--',
           lw=2,
           color='k')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.set_xlabel('$\mathrm{[Fe/H]}$',size=lab_size)
f1.set_ylabel('$\mathrm{[Mg/Fe]}$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(ymin,ymax)

f2.set_xlim(xmin,xmax)
f2.set_xticks([])
f2.set_yticks([])

f3.set_ylim(ymin,ymax)
f3.set_xticks([])
f3.set_yticks([])

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.subplots_adjust(wspace=0,hspace=0)

#plt.tight_layout()
plt.show()

## ❓QUIZ 05❓
 > **Draw gas phase-space diagram (Temperature vs. density)**  

In [ ]:
fig = plt.figure(figsize=(14,8))

f2 = plt.subplot2grid((4,5), (0, 0),fig=fig,colspan=3)
f1 = plt.subplot2grid((4,5), (1, 0),colspan=3,rowspan=3,fig=fig)
f3 = plt.subplot2grid((4,5), (1, 3),fig=fig,rowspan=3)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 
yarr = 

warr = 

bins = 400
cmap=plt.cm.jet

xmin = -5
xmax = 3

ymin = 1
ymax = 8

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr,
          weights=warr,
          range=[[xmin,xmax],
                 [ymin,ymax]],
          bins=bins,
          cmap=cmap,
          norm=colors.LogNorm())


f2.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2)
f3.hist(yarr,
        weights=warr,
        range=[ymin,ymax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2,
        orientation='horizontal')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.set_xlabel(r'$log\ \rho\ (\mathrm{H/cc})$',size=lab_size)
f1.set_ylabel('$log\ T\ (\mathrm{K})$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(ymin,ymax)

f2.set_xlim(xmin,xmax)
f2.set_xticks([])
f2.set_yticks([])

f3.set_ylim(ymin,ymax)
f3.set_xticks([])
f3.set_yticks([])

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.subplots_adjust(wspace=0,hspace=0)

#plt.tight_layout()
plt.show()

## ❓QUIZ 06❓
 > **Draw [Mg/Fe] vs. [Fe/H] diagram for gas**  

In [ ]:
FeH_c = 
MgFe_c = 

In [ ]:
fig = plt.figure(figsize=(14,8))

f2 = plt.subplot2grid((4,6), (0, 0),fig=fig,colspan=4)
f1 = plt.subplot2grid((4,6), (1, 0),colspan=4,rowspan=3,fig=fig)
f3 = plt.subplot2grid((4,6), (1, 4),fig=fig,rowspan=3)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

xarr = 
yarr = 

warr = None

bins = 200
cmap=plt.cm.magma

xmin = -1
xmax = 1

ymin = -0.5
ymax = 0.5

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.hist2d(xarr,
          yarr,
          weights=warr,
          range=[[xmin,xmax],
                 [ymin,ymax]],
          bins=bins,
          cmap=cmap,
          norm=colors.LogNorm())


f2.hist(xarr,
        weights=warr,
        range=[xmin,xmax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2)
f3.hist(yarr,
        weights=warr,
        range=[ymin,ymax],
        bins=bins,
        histtype='step',
        color='k',
        lw=2,
        orientation='horizontal')

f1.axvline(0,
           ls='--',
           lw=2,
           color='k')
f1.axhline(0,
           ls='--',
           lw=2,
           color='k')
f2.axvline(0,
           ls='--',
           lw=2,
           color='k')
f3.axhline(0,
           ls='--',
           lw=2,
           color='k')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

f1.set_xlabel('$\mathrm{[Fe/H]}$',size=lab_size)
f1.set_ylabel('$\mathrm{[Mg/Fe]}$',size=lab_size)

f1.tick_params(axis='x',labelsize=f_tick*lab_size)
f1.tick_params(axis='y',labelsize=f_tick*lab_size)

f1.set_xlim(xmin,xmax)
f1.set_ylim(ymin,ymax)

f2.set_xlim(xmin,xmax)
f2.set_xticks([])
f2.set_yticks([])

f3.set_ylim(ymin,ymax)
f3.set_xticks([])
f3.set_yticks([])

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

plt.subplots_adjust(wspace=0,hspace=0)

#plt.tight_layout()
plt.show()

## ❓QUIZ 07❓
 > **Calculate the half-mass radius & the effective radius of a galaxy**
>
>  Hint: use np.argsort(), np.where(), and np.cumsum() 

In [ ]:
r3d = np.sqrt(np.square(star['x']-ktrg['x'])+
              np.square(star['y']-ktrg['y'])+
              np.square(star['z']-ktrg['z']))

## ❓QUIZ 08❓
 > **1. Transform the coordinate from Cartesian to cylindrical coordinate**
> 
 > **2. Calculate the V/$\sigma$ of star & gaseous cell**

In [ ]:
from statsmodels.stats.weightstats import DescrStatsW

In [ ]:
## For you

def cart_to_cyl(xarr, yarr, zarr, 
                vxarr, vyarr, vzarr):
    
    r = np.sqrt(np.square(xarr)+np.square(yarr))
    phi = np.arctan2(yarr,xarr)
    
    phisort = (phi<0)
    phi[phisort] += 2*np.pi

    z = zarr

    v_r = np.cos(phi)*vxarr + np.sin(phi)*vyarr
    v_phi = -np.sin(phi)*vxarr + np.cos(phi)*vyarr
    v_z = vzarr

    return r, phi, z, v_r, v_phi, v_z
    

In [ ]:
xarr = star['x'] - ktrg['x']
yarr = star['y'] - ktrg['y']
zarr = star['z'] - ktrg['z']

vxarr = star['vx'] - ktrg['vx']
vyarr = star['vy'] - ktrg['vy']
vzarr = star['vz'] - ktrg['vz']

In [ ]:
r,phi,z,vr,vphi,vz = cart_to_cyl(xarr, yarr, zarr, 
                                 vxarr, vyarr, vzarr)

## ❓QUIZ $\infty$❓
 > **Do whatever you want to do.**
> 
 > **If you are not sure whether it is possible or not, please ask.** 